### Image Retrieval Function

In [1]:
import nuclio

In [2]:
%run ../../set_env.ipynb

In [3]:
# nuclio: start-code

In [4]:
import requests
import json
import base64
import os
import time

headers = {
            "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }

def seek_location(context,
                  shardId: int = 0,
                  container: str = '',
                  stream: str = "",
                  url = '',
                 secondsLapsed = 1):
    payload = {
            "ShardId": shardId,
            "Type": "TIME",
        "TimestampSec":int(time.time())-secondsLapsed,
        "TimestampNSec" : 0
          }
    headers["X-v3io-function"] = "Seek"
    r = requests.post(url, headers=headers,json=payload)
    return json.loads(r.text)['Location']

def get_image(context,
              url,
              shardId,
              container,
              stream,
              secondsLapsed):
    location = seek_location(context,shardId=shardId,
                             container=container,
                             stream=stream,
                             url=url,
                            secondsLapsed = secondsLapsed)
    img_payload = {
            "ShardId":    shardId,
            "Location":   location,
        "Limit":1
    }
    headers["X-v3io-function"] = "GetRecords"
    r = requests.post(url, headers=headers,json=img_payload)
    
    response=json.loads(r.text)
    
    return response

def handler(context, event):
    params = event.fields
    shardId=int(params['shardId'])
    container=params['container']
    stream=params['stream']
    
    url = "http://v3io-webapi:8081/%s/%s/"% (container,stream)
    headers = { "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
    }

    secondsLapsed = 1
    while True:
          response = get_image(context,url,shardId,container,stream,secondsLapsed )
          data = False  
          for record in response['Records']:
                data = base64.b64decode(record['Data']) 
          if data:
            break
          else:
              secondsLapsed += 1
    
    return data

In [5]:
# nuclio: end-code

In [6]:
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function(os.getenv('GET_IMAGE_FUNCTION'), kind='nuclio')

fn.with_http(workers=1)

fn.spec.base_spec['spec']['build']['baseImage'] = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/{os.getenv('DOCKER_IMAGE')}"
fn.spec.base_spec['spec']['loggerSinks'] = [{'level': 'info'}]
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1

fn.set_env('V3IO_ACCESS_KEY',os.getenv('V3IO_ACCESS_KEY'))
fn.set_env('IGZ_CONTAINER',IGZ_CONTAINER)
fn.set_env('RAW_VIDEO_STREAM',RAW_VIDEO_STREAM)
fn.set_env('TAGGED_VIDEO_STREAM',TAGGED_VIDEO_STREAM)
fn.set_env('CAMERA_LIST_TBL',CAMERA_LIST_TBL)
fn.set_env('GET_IMAGE_FUNCTION', GET_IMAGE_FUNCTION)

fn.export("../yaml/deploy-image-retrieval.yaml")

> 2020-10-27 23:13:27,598 [info] function spec saved to path: ../yaml/deploy-image-retrieval.yaml
